In [1]:
n = 5
m = 2 ** n
iterations = 4

print (f"n: {n}, m: {m}, iterations: {iterations}")

n: 5, m: 32, iterations: 4


# Theoretical error

In [2]:
import numpy as np

a = np.ones(m, dtype=complex) / np.sqrt(m)
b = np.ones(m-1, dtype=complex) / np.sqrt(m-1)
print (f"a = {abs(a.conj().T @ a)}")
print (f"b = {abs(b.conj().T @ b)}")
x = a[0]/b[0]
y = a[0]
print (f"x = {np.abs(x)}")
print (f"y = {np.abs(y)}")
phi = np.arctan(np.abs(y/x))
print (f"phi = {phi}")

for t in range(6):
    angle_t = phi * (2*t + 1)
    sin = np.sin(angle_t)
    cos = np.cos(angle_t)
    #error = cos**2/sin**2
    error = np.cos(angle_t) ** 2
    print (f"i-{t} angle: {angle_t} error: {error}")


a = 0.9999999999999999
b = 0.9999999999999999
x = 0.9842509842514762
y = 0.17677669529663687
phi = 0.17771060084511178
i-0 angle: 0.17771060084511178 error: 0.96875
i-1 angle: 0.5331318025353353 error: 0.7416992187499999
i-2 angle: 0.8885530042255589 error: 0.39757537841796853
i-3 angle: 1.2439742059157826 error: 0.10306346416473368
i-4 angle: 1.599395407606006 error: 0.0008176844567060608
i-5 angle: 1.9548166092962296 error: 0.14036333883996147


# PennyLane implementation

In [3]:
import pennylane as qml
from pennylane import numpy as np

In [4]:
# Oracle unitary operatior
U = np.eye(2**(n+1))
U = np.vstack([U[:-2],U[-2::][::-1]])
U

tensor([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 1., 0.]], requires_grad=True)

In [5]:
device = qml.device('default.qubit', shots=10000000, wires=n+1)

In [6]:
def U_start():
    qml.PauliX(wires=n)
    for i in range(n+1):
        qml.Hadamard(wires=i)

In [7]:
 def U_b():
    """Apply Oracle"""
    qml.QubitUnitary(U, wires=range(n+1))

In [8]:
 def U_c():
    for i in range(n):
        qml.Hadamard(wires=i)
        qml.PauliX(wires=i)

    qml.QubitUnitary(U, wires=range(n+1))
    for i in range(n):
        qml.PauliX(wires=i)
        qml.Hadamard(wires=i)

In [9]:
def U_iteration():
    U_b()
    U_c()

In [10]:
@qml.qnode(device)
def circuit(N: int):
    U_start()
    for t in range(N):
        U_iteration()
    return qml.probs(wires=range(n))

In [11]:
probs = circuit(N=iterations)

In [12]:
print (circuit.draw())

 0: ──H─────╭U0──H──X──╭U0──X──H──╭U0──H──X──╭U0──X──H──╭U0──H──X──╭U0──X──H──╭U0──H──X──╭U0──X──H──╭┤ Probs 
 1: ──H─────├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├┤ Probs 
 2: ──H─────├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├┤ Probs 
 3: ──H─────├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├┤ Probs 
 4: ──H─────├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──├U0──H──X──├U0──X──H──╰┤ Probs 
 5: ──X──H──╰U0────────╰U0────────╰U0────────╰U0────────╰U0────────╰U0────────╰U0────────╰U0─────────┤       
U0 =
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]



In [13]:
print (f"probs: {probs}")
print (f"errors: {1 - probs[-1]:4f}")

probs: [2.750000e-05 2.450000e-05 2.630000e-05 2.430000e-05 2.330000e-05
 2.520000e-05 2.570000e-05 3.090000e-05 2.570000e-05 2.820000e-05
 2.640000e-05 2.670000e-05 2.530000e-05 2.650000e-05 2.470000e-05
 2.660000e-05 2.530000e-05 2.590000e-05 2.460000e-05 2.570000e-05
 2.660000e-05 2.600000e-05 2.720000e-05 2.640000e-05 2.470000e-05
 2.860000e-05 2.510000e-05 2.750000e-05 2.460000e-05 2.530000e-05
 2.730000e-05 9.991914e-01]
errors: 0.000809
